You have to work on the Dogs adoptions dataset.

It contains three files:

- dogs.csv, shortly dogs
- dogTravel.csv, shortly travels
- NST-EST2021-POP.csv

Notes
1. It is mandatory to use GitHub for developing the project.
2. The project must be a jupyter notebook.
3. There is no restriction on the libraries that can be used, nor on the Python version.
4. All questions on the project must be asked in a public channel on Zulip.
5. At most 3 students can be in each group. You must create the groups by yourself.
6. You do not have to send me the project before the discussion.

# Import libraries and datasets

To carry out the project I use the libraries Pandas and Numpy

In [1]:
import pandas as pd
import numpy as np

To import the datasets I use the read_csv function

In [2]:
dogs = pd.read_csv("https://github.com/vlongo9/prova/raw/main/dogs.zip", header = 0) 
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


In [3]:
dogtravel = pd.read_csv("https://github.com/vlongo9/prova/raw/main/dogTravel.zip", header = 0) 
dogtravel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [4]:
countries = pd.read_csv("https://github.com/vlongo9/prova/raw/main/country.zip", names = ["country", "pop"]) 
countries.head()

,country,pop
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


# Check the datasets

I verify that the attribute types are as expected by possibly correcting anomalies in the import step

In [5]:
dogs.dtypes

id                   int64
org_id              object
url                 object
type.x              object
species             object
breed_primary       object
breed_secondary     object
breed_mixed           bool
breed_unknown         bool
color_primary       object
color_secondary     object
color_tertiary      object
age                 object
sex                 object
size                object
coat                object
fixed                 bool
house_trained         bool
declawed           float64
special_needs         bool
shots_current         bool
env_children        object
env_dogs            object
env_cats            object
name                object
status              object
posted              object
contact_city        object
contact_state       object
contact_zip         object
contact_country     object
stateQ              object
accessed            object
type.y              object
description         object
stay_duration        int64
stay_cost          float64
d

In [6]:
dogtravel.dtypes

index             int64
id                int64
contact_city     object
contact_state    object
description      object
found            object
manual           object
remove           object
still_there      object
dtype: object

In [7]:
countries.dtypes

country    object
pop        object
dtype: object

# Correct data type

Since operations will be required on the 'population' attribute of the countries dataset I transform it to float type (decimal number)

In [8]:
countries['pop'] = countries['pop'].str.replace(".", "", regex=True)
countries['pop'] = countries['pop'].astype(float)
countries.head()

,country,pop
0,Alabama,5024279.0
1,Alaska,733391.0
2,Arizona,7151502.0
3,Arkansas,3011524.0
4,California,39538223.0


Since operations will be required on the 'posted' and 'accessed' attributes of the dogtravel dataset I transform them to datetime objects using pd.to_datetime(). 
I also make the 'posted' column timezone-naive by removing the timezone information.

In [9]:
dogs['posted'] = pd.to_datetime(dogs['posted'], errors='coerce', format='%Y-%m-%d')
dogs['accessed'] = pd.to_datetime(dogs['accessed'],errors='coerce', format='%Y-%m-%d')

dogs["posted"] = dogs["posted"].dt.tz_localize(None)

# 1. Extract all dogs with status that is not adoptable

To extract all dogs with status that is not adoptable, I filter the rows based on the "status" column using the "!=" operator which means "different from". Then, I assign the filtered results to the "not_adoptable_dogs" dataframe.

In [10]:
not_adoptable_dogs = dogs[dogs["status"] != "adoptable"]
not_adoptable_dogs

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,NV,89146,US,89009,2019-09-20,NaT,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,AZ,85249,US,AZ,2019-09-20,NaT,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,NY,12220,US,CT,2019-09-20,NaT,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,NY,12220,US,CT,2019-09-20,NaT,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,NY,12220,US,CT,2019-09-20,NaT,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,...,NY,12477,US,CT,2019-09-20,NaT,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,...,NY,12477,US,CT,2019-09-20,NaT,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,...,VA,20136,US,DC,2019-09-20,NaT,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,...,MD,20905,US,DC,2019-09-20,NaT,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,...,PA,17325,US,DC,2019-09-20,NaT,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


# 2. For each (primary) breed, determine the number of dogs

To determine the number of dogs for each primary breed I firstly use the "groupby()" function that groups the dogs based on the "breed_primary" column. Then, the "count()" function counts the number of non-null values in the "id" column for each breed_primary group. The "breed_counts" dataframe shows us the results.

In [11]:
breed_counts = dogs.groupby("breed_primary")["id"].count().reset_index(name="count")
breed_counts

,breed_primary,count
0,Affenpinscher,17
1,Afghan Hound,4
2,Airedale Terrier,19
3,Akbash,3
4,Akita,181
...,...,...
211,Wirehaired Pointing Griffon,1
212,Wirehaired Terrier,60
213,Xoloitzcuintli / Mexican Hairless,11
214,Yellow Labrador Retriever,158


# 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. 

To determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed, I firstly group the dogs based on both "breed_primary" and "breed_mixed" columns. Then, I count the number of occurrences of the "id" column within each combination of primary breed and mixed breed status. Lastly, I create a variable called "ratio" which contains the values of the ratios between the number of the dogs of Mixed Breed and those not of Mixed Breed. THe results are shown in the breed_counts_pivot dataframe.

In [12]:
breed_counts = dogs.groupby(["breed_primary", "breed_mixed"])["id"].count().reset_index(name="count")

breed_counts_pivot = breed_counts.pivot(index="breed_primary", columns="breed_mixed", values="count")

breed_counts_pivot["ratio"] = breed_counts_pivot[True] / breed_counts_pivot[False]
breed_counts_pivot

breed_mixed,False,True,ratio
breed_primary,,,
Affenpinscher,12.0,5.0,0.416667
Afghan Hound,NaN,4.0,NaN
Airedale Terrier,2.0,17.0,8.500000
Akbash,1.0,2.0,2.000000
Akita,98.0,83.0,0.846939
...,...,...,...
Wirehaired Pointing Griffon,NaN,1.0,NaN
Wirehaired Terrier,15.0,45.0,3.000000
Xoloitzcuintli / Mexican Hairless,6.0,5.0,0.833333


# 4. For each (primary) breed, determine the earliest and the latest posted timestamp

To determine the earliest and the latest posted timestamp for each primary breed, I firstly group the dogs based on the "breed_primary" column and then aggregate the "posted" column using the agg() function. The agg() function is used to apply multiple aggregation functions, in this case, min and max, to the "posted" column for each group. Finally, the code displays the resulting dataframe breed_posted. It contains the primary breed, the earliest posted date and the latest posted date for each breed.

In [13]:
breed_posted = dogs.groupby("breed_primary")["posted"].agg([min, max]).reset_index()

breed_posted.columns = ["breed_primary", "earliest_posted", "latest_posted"]
breed_posted

,breed_primary,earliest_posted,latest_posted
0,Affenpinscher,2012-03-08 10:27:33,2019-09-14 10:10:51
1,Afghan Hound,2017-06-29 23:28:51,2019-07-27 00:38:48
2,Airedale Terrier,2014-06-13 12:59:36,2019-09-19 18:40:39
3,Akbash,2019-07-21 00:35:59,2019-08-23 17:11:04
4,Akita,2012-03-03 09:31:08,2019-09-20 15:19:57
...,...,...,...
211,Wirehaired Pointing Griffon,2016-06-29 20:03:55,2016-06-29 20:03:55
212,Wirehaired Terrier,2012-11-27 14:07:54,2019-09-19 22:52:45
213,Xoloitzcuintli / Mexican Hairless,2007-02-01 00:00:00,2019-09-08 11:15:54
214,Yellow Labrador Retriever,2010-05-31 00:00:00,2019-09-20 06:30:27


# 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

Firstly I group the dogs based on the "contact_state" column. Then count the occurrences of each sex (male and female) within each state using the value_counts() function. The unstack() function is used to reshape the data, so that each sex becomes a separate column. After replacing any NaN values with 0 (ensuring that all cells have a value), I calculate the imbalance between the number of males and females for each state. I create a new column "imbalance" in the state_imbalance DataFrame, where the value is obtained by subtracting the count of females from the count of males.

In [14]:
state_imbalance = dogs.groupby("contact_state")["sex"].value_counts().unstack().reset_index()

state_imbalance = state_imbalance.fillna(0)

state_imbalance["imbalance"] = state_imbalance["Male"] - state_imbalance["Female"]

state_imbalance_sorted = state_imbalance.sort_values('imbalance', ascending=False)
state_imbalance_sorted

sex,contact_state,Female,Male,Unknown,imbalance
58,OH,1231.0,1439.0,0.0,208.0
37,IN,850.0,1027.0,0.0,177.0
69,VA,1450.0,1608.0,0.0,158.0
42,MD,679.0,814.0,0.0,135.0
66,TN,825.0,944.0,0.0,119.0
...,...,...,...,...,...
53,NH,172.0,163.0,0.0,-9.0
29,DC,176.0,160.0,0.0,-16.0
43,ME,287.0,258.0,0.0,-29.0
47,MS,275.0,235.0,0.0,-40.0


In [15]:
largest_imbalance = state_imbalance.loc[abs(state_imbalance["imbalance"]).idxmax()]

print("The state with the largest sex imbalance is", largest_imbalance["contact_state"], 
      "with a sex imbalance of", largest_imbalance["imbalance"])

The state with the largest sex imbalance is OH with a sex imbalance of 208.0


# 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay

The code used to obtain the average duration of the stay and the average cost of stay for each pair (age, size) firstly groups the dogs based on both the "age" and "size" columns. It then calculates the mean (average) values of the "stay_duration" and "stay_cost" columns within each combination of age and size groups using the agg() function. Lastly it renames the columns generated by the agg() function to better visualize the results.

In [16]:
age_size_stats = dogs.groupby(["age", "size"]).agg({"stay_duration": "mean", "stay_cost": "mean"}).reset_index()

age_size_stats.columns = ["age", "size", "avg_stay_duration", "avg_stay_cost"]
age_size_stats

,age,size,avg_stay_duration,avg_stay_cost
0,Adult,Extra Large,89.015414,232.591561
1,Adult,Large,89.531943,238.661141
2,Adult,Medium,89.421036,238.258977
3,Adult,Small,89.407479,238.974838
4,Baby,Extra Large,87.032967,237.180879
5,Baby,Large,89.701564,238.698827
6,Baby,Medium,89.577668,237.108131
7,Baby,Small,89.958291,239.083810
8,Senior,Extra Large,88.861111,235.232361
9,Senior,Large,88.984298,237.507364


# 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs

It is necessary to merge the two dataframes dogs and dogtravel. The following code combines the rows from both DataFrames that have matching "id" values into a new DataFrame called dog_travel_merge.

In [17]:
dog_travel_merge = dogs.merge(dogtravel, on="id")
dog_travel_merge.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,stay_duration,stay_cost,index,contact_city_y,contact_state_y,description_y,found,manual,remove,still_there
0,45923603,NV173,https://www.petfinder.com/dog/blair-45923603/n...,Dog,Dog,German Shepherd Dog,NaN,True,False,NaN,...,79,217.36,389,Las Vegas,NV,Meet Blair. Blair was confiscated from a hoard...,Arizona,NaN,NaN,NaN
1,45923600,NV173,https://www.petfinder.com/dog/duane-45923600/n...,Dog,Dog,German Shepherd Dog,NaN,True,False,NaN,...,104,152.50,390,Las Vegas,NV,Handsome Duane was confiscated from part of a ...,Arizona,NaN,NaN,NaN
2,45289127,NV162,https://www.petfinder.com/dog/mustard-45289127...,Dog,Dog,Beagle,NaN,False,False,NaN,...,132,238.26,2100,Las Vegas,NV,Mustard was part of a pair. He came in with Br...,Las Vegas,Texas,NaN,NaN
3,45289127,NV162,https://www.petfinder.com/dog/mustard-45289127...,Dog,Dog,Beagle,NaN,False,False,NaN,...,132,238.26,4704,Las Vegas,NV,Mustard was part of a pair. He came in with Br...,Texas,NaN,NaN,NaN
4,44801851,NV205,https://www.petfinder.com/dog/max-44801851/nv/...,Dog,Dog,Boxer,NaN,True,False,Bicolor,...,76,159.10,3798,Las Vegas,NV,Our Max.... This boy has been through a lot an...,Puerto Rico,NaN,NaN,NaN


To find the dogs involved in at least 3 travels, I firstly group and count the dogs based on the "id" columns ant then filter the values to extract the "id" values where the count of unique values in the "index" column is greater than or equal to 3. Visualizing the results I add the column "breed_primary" that shows us also the breed of the dog in question.

In [18]:
travel_counts = dog_travel_merge.groupby("id")["index"].nunique().reset_index()

ids_3travels = travel_counts.loc[travel_counts["index"] >= 3, "id"]

Finally, it is necessary to filter the dogs DataFrame based on the "id" values in ids_3travels to select only the rows where the ID is present in ids_3travels and select only the "id" and "breed_primary" columns from the filtered rows. The drop_duplicates() function is used to remove any duplicate rows in the resulting dogs_3travels DataFrame.

In [19]:
dogs_3travels = dogs.loc[dogs["id"].isin(ids_3travels), ["id", "breed_primary"]].drop_duplicates()

dogs_3travels

,id,breed_primary
1159,45642530,Jindo
6835,46039420,Border Collie
8526,40036107,Pit Bull Terrier
10681,45851842,Labrador Retriever
10803,45841145,Mixed Breed
...,...,...
56850,41144335,Chihuahua
56864,40103682,Rat Terrier
56875,38664932,Pit Bull Terrier
56879,38495992,Pit Bull Terrier


# 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found

First of all let's fill any missing values in the "found" and "manual" columns of the dogtravel DataFrame with an empty string (""), using the fillna() function. This ensures that there are no null or NaN values in those columns.

In [20]:
dogtravel ["found"] = dogtravel["found"].fillna("")

dogtravel ["manual"] = dogtravel["manual"].fillna("")

Then, I create a new column called "correct_state" and assign values to it.
I use the apply() function to apply a lambda function to each row of the DataFrame. The lambda function checks if the value in the "manual" column is not an empty string. If it is not empty, it assigns the value from the "manual" column to the "Correct_state" column for that row. Otherwise, it splits the value in the "found" column by commas, selects the last element of the resulting list using [-1], and removes any leading or trailing whitespace using strip(). This value is then assigned to the "correct_state" column for that row.

In [21]:
dogtravel["correct_state"] = dogtravel.apply(lambda x: x["manual"] if x["manual"] != "" else x["found"].split(",")[-1].strip(), axis=1)

dogtravel

,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,,NaN,NaN,Arkansas
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN,Bahamas
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN,Maryland
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,,True,NaN,Adaptil
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,,NaN,NaN,Afghanistan
...,...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,,True,NaN,WV
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,,NaN,NaN,Wyoming
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,,True,NaN,Yazmin
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Young,Ohio,NaN,NaN,Ohio


# 9. For each state, compute the ratio between the number of travels and the population

Let's firstly consider the dogtravel dataframe. It is necessary to obtain a new column in the dogtravel dataset that contains the number of travels for each state. To do that, I group the DataFrame by contact state and count the number of rows in each group. Then I create a new DataFrame named state_counts_df that contains the state counts as a column and merge it with the original dataframe called dogtravel.

In [22]:
state_counts = dogtravel.groupby('contact_state').size()

state_counts_df = pd.DataFrame({'numberoftravels': state_counts})

dogtravel_new = pd.merge(dogtravel, state_counts_df, how='left', on='contact_state')

dogtravel_new


,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state,numberoftravels
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,,NaN,NaN,Arkansas,190
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN,Bahamas,133
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN,Maryland,379
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,,True,NaN,Adaptil,190
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,,NaN,NaN,Afghanistan,103
...,...,...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,,True,NaN,WV,27
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,,NaN,NaN,Wyoming,66
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,,True,NaN,Yazmin,109
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Young,Ohio,NaN,NaN,Ohio,177


The second step is to define a mapping between state abbreviations and names and replace the state abbreviations with their names in the dogtravel_new table (this procedure is required as the countries dataframe contains a columns with the names of the states and not the abbreviations).

In [23]:
state_mapping = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

dogtravel_new['contact_state'] = dogtravel_new['contact_state'].replace(state_mapping)

Let's now merge the resulting dataframe with the country dataset on contact_state field.

In [24]:
merged = dogtravel_new.merge(countries, left_on='contact_state', right_on='country')

merged

,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state,numberoftravels,country,pop
0,0,44520267,Anoka,Minnesota,Boris is a handsome mini schnauzer who made hi...,Arkansas,,NaN,NaN,Arkansas,190,Minnesota,5706494.0
1,3,44475904,Saint Cloud,Minnesota,~~Came in to the shelter as a transfer from an...,Adaptil,,True,NaN,Adaptil,190,Minnesota,5706494.0
2,312,45419523,Minneapolis,Minnesota,You can fill out an adoption application onlin...,Alabama,,NaN,NaN,Alabama,190,Minnesota,5706494.0
3,405,45502015,Blaine,Minnesota,Floyd is back and looking for his forever home...,Arizona,,NaN,NaN,Arizona,190,Minnesota,5706494.0
4,436,45724780,Plymouth,Minnesota,You can fill out an adoption application onlin...,Arkansas,,True,NaN,Arkansas,190,Minnesota,5706494.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6065,2176,45906452,New Iberia,Louisiana,Will you allow me to be your shadow? I just gr...,Louisiana,,NaN,True,Louisiana,5,Louisiana,4657757.0
6066,3260,44264372,Natchitoches,Louisiana,I was found in May of 2012 across from Nicky's...,Nicky,,True,NaN,Nicky,5,Louisiana,4657757.0
6067,3704,46008478,Baton Rouge,Louisiana,Peanut was found wandering down a busy street...,Peanut,,True,NaN,Peanut,5,Louisiana,4657757.0
6068,4044,40700634,Baker,Louisiana,Meet YoYo! This beautiful shepherd mix girl i...,S.,Louisiana,NaN,NaN,Louisiana,5,Louisiana,4657757.0


Finally, I calculate the ratio of number/population and add it as a new column in the merged dataframe.

In [25]:
merged['ratio'] = merged['numberoftravels'] / merged['pop']

merged[['country','numberoftravels', 'pop','ratio']].drop_duplicates()


,country,numberoftravels,pop,ratio
0,Minnesota,190,5706494.0,3.329540e-05
190,Florida,133,21538187.0,6.175079e-06
323,Maryland,379,6177224.0,6.135442e-05
702,Colorado,103,5773714.0,1.783947e-05
805,Connecticut,90,3605944.0,2.495879e-05
895,Ohio,177,11799448.0,1.500070e-05
1072,Alabama,75,5024279.0,1.492751e-05
1147,New York,490,20201249.0,2.425593e-05
1637,New Jersey,552,9288994.0,5.942516e-05
2189,Pennsylvania,316,13002700.0,2.430264e-05


# 10. For each dog, compute the number of days from the posted day to the day of last access

In [26]:
from datetime import datetime

I can calculate time difference between "accessed" and "posted" columns. By dividing the time difference by the timedelta(days=1) I obtain the difference expressed in days.

In [27]:
dogs["time_diff"] = dogs["accessed"] - dogs["posted"]

dogs["days_diff"] = dogs["time_diff"] / pd.Timedelta(days=1)

dogs[["id","accessed","posted","days_diff"]]

,id,accessed,posted,days_diff
0,46042150,2019-09-20,2019-09-20 16:37:59,-0.693044
1,46042002,2019-09-20,2019-09-20 16:24:57,-0.683993
2,46040898,2019-09-20,2019-09-20 14:10:11,-0.590405
3,46039877,2019-09-20,2019-09-20 10:08:22,-0.422477
4,46039306,2019-09-20,2019-09-20 06:48:30,-0.283681
...,...,...,...,...
58175,44605893,2019-09-20,2019-05-03 14:23:49,139.400127
58176,44457061,2019-09-20,2019-04-13 16:20:24,159.319167
58177,42865848,2019-09-20,2018-09-27 04:18:56,357.820185
58178,42734734,2019-09-20,2018-09-12 05:03:38,372.789144


# 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

Using the variable obtained in the previous point, I convert the difference expressed in days into the one expressed in weeks.

In [28]:
dogs["weeks_diff"] = dogs["time_diff"] / pd.Timedelta(weeks=1)

dogs[["id","accessed","posted","weeks_diff"]]

,id,accessed,posted,weeks_diff
0,46042150,2019-09-20,2019-09-20 16:37:59,-0.099006
1,46042002,2019-09-20,2019-09-20 16:24:57,-0.097713
2,46040898,2019-09-20,2019-09-20 14:10:11,-0.084344
3,46039877,2019-09-20,2019-09-20 10:08:22,-0.060354
4,46039306,2019-09-20,2019-09-20 06:48:30,-0.040526
...,...,...,...,...
58175,44605893,2019-09-20,2019-05-03 14:23:49,19.914304
58176,44457061,2019-09-20,2019-04-13 16:20:24,22.759881
58177,42865848,2019-09-20,2018-09-27 04:18:56,51.117169
58178,42734734,2019-09-20,2018-09-12 05:03:38,53.255592


Then, I define the partition criteria and use pd.cut() to create a new column that partitions the dogs based on the number of weeks difference.

In [29]:
bins = [-float('inf'), 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, float('inf')]
labels = ['0-2', '2-4', '4-6', '6-8', '8-10', '10-12','12-14', '14-16','16-18', '18-20', '20+']

dogs["weeks_partition"] = pd.cut(dogs["weeks_diff"], bins=bins, labels=labels)

dogs[["id","accessed","posted","weeks_diff", "weeks_partition"]]

,id,accessed,posted,weeks_diff,weeks_partition
0,46042150,2019-09-20,2019-09-20 16:37:59,-0.099006,0-2
1,46042002,2019-09-20,2019-09-20 16:24:57,-0.097713,0-2
2,46040898,2019-09-20,2019-09-20 14:10:11,-0.084344,0-2
3,46039877,2019-09-20,2019-09-20 10:08:22,-0.060354,0-2
4,46039306,2019-09-20,2019-09-20 06:48:30,-0.040526,0-2
...,...,...,...,...,...
58175,44605893,2019-09-20,2019-05-03 14:23:49,19.914304,18-20
58176,44457061,2019-09-20,2019-04-13 16:20:24,22.759881,20+
58177,42865848,2019-09-20,2018-09-27 04:18:56,51.117169,20+
58178,42734734,2019-09-20,2018-09-12 05:03:38,53.255592,20+


# 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.


To solve the last point, it is necessary to import the following libraries.

In [30]:
from difflib import SequenceMatcher
import pandas as pd

Then, I add a new column named 'description_similarity' to describe the similarity.

To satisfy the first condition of similarity I use the groupby() function that groups the data based on the columns breed_primary', 'breed_secondary', and 'sex'.
The transform() function applies a transformation on each group separately. Within the transformation, the lambda function is used to calculate the similarity ratio for each description within a group.
The inner lambda function compares the descriptions using the SequenceMatcher class and calculates the similarity ratio using the ratio() method.
If the length of the group is greater than 1, it means there are multiple descriptions to compare, so the similarity ratio is calculated.
If the length of the group is 1, it means there is only one description, so the similarity ratio is set to 0.0 if the description is of float type (to avoid TypeError), otherwise set to 0.0.

The second line filters the dogs DataFrame to include only the rows where the 'description_similarity' column is greater than or equal to 0.9, indicating a high similarity.
The resulting DataFrame is then sorted in descending order based on the 'description_similarity' column.

In [31]:
dogs['description_similarity'] = dogs.groupby(['breed_primary', 'breed_secondary', 'sex'])['description'].transform(lambda x: x.apply(lambda d: SequenceMatcher(None, str(x.iloc[0]), str(d)).ratio()) if len(x) > 1 else 0.0 if isinstance(x.iloc[0], float) else 0.0)

# Find duplicate records based on the defined criteria
duplicates = dogs[(dogs['description_similarity'] >= 0.9)].sort_values('description_similarity', ascending=False)

# Display the duplicate records
duplicates


,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,accessed,type.y,description,stay_duration,stay_cost,time_diff,days_diff,weeks_diff,weeks_partition,description_similarity
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,-1 days +07:22:01,-0.693044,-0.099006,0-2,1.000000
24114,45279120,FL1002,https://www.petfinder.com/dog/levi-45279120/fl...,Dog,Dog,Corgi,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,2019-09-20,Dog,Levi is available for foster or adoption. We a...,83,267.98,64 days 10:54:50,64.454745,9.207821,8-10,1.000000
23960,45428154,FL426,https://www.petfinder.com/dog/pheobe-45428154/...,Dog,Dog,Labrador Retriever,Mixed Breed,True,False,NaN,...,2019-09-20,Dog,NaN,87,285.90,50 days 06:32:53,50.272836,7.181834,6-8,1.000000
23963,45428136,FL426,https://www.petfinder.com/dog/cruiser-45428136...,Dog,Dog,American Bulldog,Mixed Breed,True,False,NaN,...,2019-09-20,Dog,NaN,120,529.68,50 days 06:32:53,50.272836,7.181834,6-8,1.000000
23966,45428013,FL426,https://www.petfinder.com/dog/stewart-45428013...,Dog,Dog,Hound,Mixed Breed,True,False,NaN,...,2019-09-20,Dog,NaN,95,412.05,50 days 06:32:53,50.272836,7.181834,6-8,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21077,45735578,AL186,https://www.petfinder.com/dog/eerie-45735578/p...,Dog,Dog,Labrador Retriever,Spaniel,True,False,Black,...,2019-09-20,Dog,EERIE\nFEMALE\nLAB SPANIEL MIX\nAGE 15 WEEKS\n...,77,77.87,26 days 03:02:18,26.126597,3.732371,2-4,0.903984
17219,45735782,AL186,https://www.petfinder.com/dog/eerie-45735782/m...,Dog,Dog,Labrador Retriever,Spaniel,True,False,Black,...,2019-09-20,Dog,EERIE\nFEMALE\nLAB SPANIEL MIX\nAGE 15 WEEKS\n...,59,285.36,26 days 02:59:43,26.124803,3.732115,2-4,0.903522
48728,45190927,OK51,https://www.petfinder.com/dog/piper-45190927/o...,Dog,Dog,Shar-Pei,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,2019-09-20,Dog,This beautiful girl is Piper :) She was found ...,72,81.59,74 days 18:19:25,74.763484,10.680498,10-12,0.903315
20828,45864102,AL186,https://www.petfinder.com/dog/aiden-b-45864102...,Dog,Dog,Labrador Retriever,Golden Retriever,True,False,NaN,...,2019-09-20,Dog,"Meet Aiden... he is a great boy, he is about 5...",88,239.95,15 days 04:14:32,15.176759,2.168108,2-4,0.901077
